In [1]:
!pip install -r requirements.txt

In [2]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras import Sequential
import os,cv2,random
import numpy as np

2025-01-19 10:01:03.117900: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
categories = ['without_mask', 'with_mask']

In [4]:
categories = ['without_mask', 'with_mask']
data = []

for category in categories:
    path = os.path.join('./data', category)
    label = categories.index(category)

    for file in os.listdir(path):
        
        if file.endswith(".jpg"):
            img_path = os.path.join(path, file)
            img = cv2.imread(img_path)
            img = cv2.resize(img,(224, 224))
        else:
            pass
    
        data.append([img, label])


In [5]:
len(data)

2874

In [6]:
random.shuffle(data)

In [7]:
X = []
y = []
for features, label in data:
    X.append(features)
    y.append(label)

In [8]:
len(X)

2874

In [9]:
len(y)

2874

In [10]:
X = np.array(X)
y = np.array(y)

In [11]:
X = X/255

In [12]:
print(X.shape)
print(y.shape)

(2874, 224, 224, 3)
(2874,)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [14]:
X_train.shape


(2299, 224, 224, 3)

In [15]:
vgg = VGG16()

2025-01-19 10:01:20.374006: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-19 10:01:20.739697: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-19 10:01:20.928814: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-19 10:01:21.156176: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2025-01-19 10:01:21.212461: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


In [16]:
model = Sequential()

In [17]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [19]:
for layer in model.layers:
    layer.trainable = False

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [21]:
model.add(Dense(1, activation= 'sigmoid'))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [23]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
model.fit(X_train, y_train, epochs=1, validation_data= (X_test, y_test))

72/72 [==============================] - 519s 7s/step - loss: 0.5648 - accuracy: 0.6842 - val_loss: 0.3936 - val_accuracy: 0.8557


In [25]:
cap = cv2.VideoCapture(0)

[ WARN:0@588.464] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@588.470] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [26]:
cap = cv2.VideoCapture(0)

def detect_Face_mask(img):

    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0


    y_pred = model.predict(img_normalized.reshape(1, 224, 224, 3))

    return y_pred

threshold = 0.716

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (224, 224))
    y_pred = detect_Face_mask(frame)


    if y_pred >= threshold:
        draw_label(frame, "Mask", (30, 30), (0, 255, 0))
    else:
        draw_label(frame, "No Mask", (30, 30), (0, 0, 255))

    print(y_pred)
    cv2.imshow("window", frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break


cap.release()


cv2.destroyAllWindows()


[ WARN:0@588.506] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@588.507] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [27]:
def detect_Face_mask(img):

   y_pred =  model.predict(img.reshape(1, 224, 224, 3))

   return y_pred

In [28]:
def draw_label(img, text, pos, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_COMPLEX, 1, cv2.FILLED)
    end_x = pos[0]+ text_size[0][0]+ 2
    end_y = pos[0]+ text_size[0][1]- 2

    cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED )
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_COMPLEX, 1,( 0, 0, 0), 1, cv2.LINE_AA)

In [29]:
# Resize the image to 224x224
image_path = './data/with_mask/with_mask_15.jpg'
sample1 = cv2.imread(image_path)  # Read the image
sample1_resized = cv2.resize(sample1, (224, 224))  # Resize image to 224x224

# Preprocess the image (normalize if needed, optional)
sample1_preprocessed = sample1_resized / 255.0  # Normalize the pixel values between 0 and 1

# Add an extra dimension for batch size, reshape for the model input: (1, 224, 224, 3)
sample1_input = sample1_preprocessed.reshape(1, 224, 224, 3)

# Prediction
y_pred = model.predict(sample1_input)

print(y_pred)


1/1 [==============================] - 1s 540ms/step
[[0.6243762]]
